# Minimal Example to Show Numba Bug in Foolbox

See issue at: 
https://github.com/bethgelab/foolbox/issues/545

In [1]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import layers
import foolbox as fb

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = (x_train.reshape(60000, 784).astype('float32') / 255)
x = tf.convert_to_tensor(tf.expand_dims(x_train[422].reshape(28,28,1), axis=0), tf.float32)*1
y = tf.convert_to_tensor([y_train[422]])*1


class CustomLayer(layers.Layer):

    def __init__(self, units=32, activation='relu', input_shape=(784)):
        super(CustomLayer, self).__init__()
        self.units = units
        self.activation = activation
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True,
                                 name='unpruned-weights')
        self.mask = self.add_weight(shape=(self.w.shape),
                                    initializer='ones',
                                    trainable=False,
                                   name='pruning-masks')
        self.pruned_w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='ones',
                                 trainable=False,
                                       name='pruned-weights')

   
    def call(self, inputs):
        self.pruned_w = tf.multiply(self.w, self.mask)
        x = tf.matmul(inputs, self.pruned_w)
        
        if self.activation == 'relu':
            return tf.keras.activations.relu(x)
        if self.activation == 'softmax':
            return tf.keras.activations.softmax(x)
        if self.activation == None:
            return x
            
        raise ValueError('Activation function not implemented')


class LeNet300_100(tf.keras.Model):
    def __init__(self):
        super(LeNet300_100, self).__init__()
        self.dense1 = CustomLayer(300, input_shape=(None, 784))
        self.dense2 = CustomLayer(100, input_shape=(None, 300))
        self.dense3 = CustomLayer(10, activation=None, input_shape=(None, 100))
        
    def call(self, inputs):
        x = tf.keras.layers.Flatten()(inputs)
        x = self.dense1(x)
        
        x = self.dense2(x)
        x = self.dense3(x)
        self.pre_softmax = x
        return tf.keras.activations.softmax(x)

def initialize_model():
    model = LeNet300_100()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,
                  metrics=['accuracy'],
                  experimental_run_tf_function=False
                 )
    return model

def train_model(model):
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    model.fit(x=x_train,
              y=y_train,
              batch_size=64,
              epochs=500,
              callbacks=[callback],
              validation_data=(x_test, y_test),
             )
    
model = initialize_model()
train_model(model)


#model = tf.keras.models.load_model('../saved-models/attack-test-model')
fmodel = fb.models.TensorFlowModel(model, bounds=(0,1))

attack = fb.attacks.L0BrendelBethgeAttack()

adversarials = attack(
    fmodel,
    x,
    y,
    epsilons=None
)

/Users/florianmerkle/dev/foolbox/foolbox/attacks/brendel_bethge.py:781: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  @jitclass(spec=[])
/Users/florianmerkle/dev/foolbox/foolbox/attacks/brendel_bethge.py:1364: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  @jitclass(spec=spec)
/Users/florianmerkle/dev/foolbox/foolbox/attacks/brendel_bethge.py:1560: NumbaDeprecationWarning: The 'numba.jitclass' decorator 

Epoch 1/500
932/938 [============================>.] - ETA: 0s - loss: 1.5741 - accuracy: 0.9016

TypeError: in user code:

    /Users/florianmerkle/anaconda3/envs/master-thesis/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:941 test_function  *
        outputs = self.distribute_strategy.run(
    <ipython-input-1-ebd2b5926635>:55 call  *
        x = self.dense1(x)
    <ipython-input-1-ebd2b5926635>:34 call  *
        x = tf.matmul(inputs, self.pruned_w)
    /Users/florianmerkle/anaconda3/envs/master-thesis/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:180 wrapper  **
        return target(*args, **kwargs)
    /Users/florianmerkle/anaconda3/envs/master-thesis/lib/python3.8/site-packages/tensorflow/python/ops/math_ops.py:2983 matmul
        return gen_math_ops.mat_mul(
    /Users/florianmerkle/anaconda3/envs/master-thesis/lib/python3.8/site-packages/tensorflow/python/ops/gen_math_ops.py:5585 mat_mul
        _, _, _op, _outputs = _op_def_library._apply_op_helper(
    /Users/florianmerkle/anaconda3/envs/master-thesis/lib/python3.8/site-packages/